In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import Dataset, DataLoader

Bagian ini mengimpor pustaka-pustaka yang diperlukan:

- torch dan torch.nn: Pustaka utama PyTorch untuk deep learning
- pandas dan numpy: Untuk manipulasi data dan operasi numerik
- Komponen sklearn: Untuk pra-pemrosesan data
- Dataset dan DataLoader: Kelas PyTorch untuk penanganan data yang efisien

In [2]:
# Custom Dataset class for handling our data
class BankDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


Kelas ini mengubah data menjadi tensor PyTorch:

- FloatTensor untuk fitur (X) karena jaringan saraf bekerja dengan angka desimal
- LongTensor untuk label (y) karena klasifikasi menggunakan label kelas berupa bilangan bulat

In [3]:
# MLP Model class with flexible architecture
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, neurons_per_layer, activation_func):
        super(MLP, self).__init__()

        # Create layer list to hold all layers
        layers = []

        # Input layer to first hidden layer
        layers.append(nn.Linear(input_size, neurons_per_layer))
        layers.append(self._get_activation(activation_func))

        # Additional hidden layers
        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(self._get_activation(activation_func))

        # Output layer
        layers.append(nn.Linear(neurons_per_layer, 2))  # Binary classification

        # Combine all layers into sequential model
        self.model = nn.Sequential(*layers)

    def _get_activation(self, name):
        activations = {
            'linear': nn.Identity(),
            'relu': nn.ReLU(),
            'sigmoid': nn.Sigmoid(),
            'tanh': nn.Tanh()
        }
        return activations.get(name.lower(), nn.ReLU())

    def forward(self, x):
        return self.model(x)


Ini mendefinisikan kelas Multi-Layer Perceptron dengan parameter yang fleksibel:

- input_size: Jumlah fitur input
- hidden_layers: Jumlah layer tersembunyi (1, 2, atau 3 dalam eksperimen kita)
- neurons_per_layer: Jumlah neuron di setiap layer tersembunyi
- activation_func: Jenis fungsi aktivasi yang digunakan

In [4]:
def preprocess_data(data):
    # Convert categorical variables to numerical
    le = LabelEncoder()
    categorical_columns = ['job', 'marital', 'education', 'default', 'housing',
                         'loan', 'contact', 'month', 'poutcome', 'y']

    for col in categorical_columns:
        data[col] = le.fit_transform(data[col])

    # Separate features and target
    X = data.drop('y', axis=1).values
    y = data['y'].values

    # Scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

Ini membangun arsitektur jaringan secara dinamis:

- Layer pertama menghubungkan fitur input ke layer tersembunyi pertama
- Setiap layer tersembunyi memiliki jumlah neuron yang sama
- Fungsi aktivasi disisipkan di antara layer linear


In [5]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return total_loss / len(train_loader), 100 * correct / total

Loop pelatihan melaksanakan langkah-langkah penting:

- Mengatur model ke mode pelatihan
- Memproses batch data melalui jaringan
- Menghitung loss dan memperbarui bobot melalui backpropagation
- Melacak statistik akurasi dan loss
- Pemanggilan model.train() penting karena mengaktifkan perhitungan gradien

In [6]:

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return total_loss / len(test_loader), 100 * correct / total

Fungsi ini:

- Mengatur model ke mode evaluasi
- Menonaktifkan perhitungan gradien untuk efisiensi
- Menghitung loss dan akurasi pada data uji
- Menggunakan torch.no_grad() untuk menghemat memori saat inferensi

In [9]:
def run_experiment(config):
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load and preprocess data
    data = pd.read_csv('/content/sample_data/bank-full.csv', sep=';')
    X, y = preprocess_data(data)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create data loaders
    train_dataset = BankDataset(X_train, y_train)
    test_dataset = BankDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])

    # Initialize model
    model = MLP(
        input_size=X.shape[1],
        hidden_layers=config['hidden_layers'],
        neurons_per_layer=config['neurons'],
        activation_func=config['activation']
    ).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

    # Training loop
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(config['epochs']):
        train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{config["epochs"]}], '
                  f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
                  f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

    return {
        'final_train_acc': train_accuracies[-1],
        'final_test_acc': test_accuracies[-1],
        'train_losses': train_losses,
        'test_losses': test_losses,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies
    }

Ini mengatur seluruh proses pelatihan:

- Memilih GPU secara otomatis jika tersedia
- Membuat data loader dengan ukuran batch yang ditentukan
- Menginisialisasi model dengan konfigurasi yang diberikan
- Menjalankan loop pelatihan untuk jumlah epoch yang ditentukan
- Mencatat dan mengembalikan metrik kinerja

In [10]:
# Experiment configurations
hidden_layers = [1, 2, 3]
neurons = [4, 8, 16, 32, 64]
activations = ['linear', 'sigmoid', 'relu', 'tanh']
epochs = [1, 10, 25, 50, 100, 250]
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128, 256, 512]

# Store results
results = []

# Run base experiment
base_config = {
    'hidden_layers': 2,
    'neurons': 32,
    'activation': 'relu',
    'epochs': 50,
    'learning_rate': 0.001,
    'batch_size': 64
}

print("Running base configuration...")
base_results = run_experiment(base_config)
results.append({
    'config': base_config,
    'results': base_results
})

# Example of running experiments for different hidden layers
print("\nTesting different numbers of hidden layers...")
for n_layers in hidden_layers:
    config = base_config.copy()
    config['hidden_layers'] = n_layers
    results.append({
        'config': config,
        'results': run_experiment(config)
    })

# Print summary of results
print("\nSummary of Results:")
for result in results:
    config = result['config']
    metrics = result['results']
    print(f"\nConfiguration:")
    print(f"Hidden Layers: {config['hidden_layers']}")
    print(f"Neurons: {config['neurons']}")
    print(f"Activation: {config['activation']}")
    print(f"Epochs: {config['epochs']}")
    print(f"Learning Rate: {config['learning_rate']}")
    print(f"Batch Size: {config['batch_size']}")
    print(f"Final Test Accuracy: {metrics['final_test_acc']:.2f}%")

Running base configuration...
Epoch [10/50], Train Loss: 0.2213, Train Acc: 90.52%, Test Loss: 0.2353, Test Acc: 89.72%
Epoch [20/50], Train Loss: 0.2128, Train Acc: 90.86%, Test Loss: 0.2318, Test Acc: 90.10%
Epoch [30/50], Train Loss: 0.2086, Train Acc: 90.91%, Test Loss: 0.2294, Test Acc: 90.28%
Epoch [40/50], Train Loss: 0.2037, Train Acc: 91.22%, Test Loss: 0.2333, Test Acc: 89.90%
Epoch [50/50], Train Loss: 0.1996, Train Acc: 91.30%, Test Loss: 0.2316, Test Acc: 90.14%

Testing different numbers of hidden layers...
Epoch [10/50], Train Loss: 0.2276, Train Acc: 90.23%, Test Loss: 0.2387, Test Acc: 90.01%
Epoch [20/50], Train Loss: 0.2223, Train Acc: 90.43%, Test Loss: 0.2365, Test Acc: 89.90%
Epoch [30/50], Train Loss: 0.2195, Train Acc: 90.56%, Test Loss: 0.2355, Test Acc: 89.91%
Epoch [40/50], Train Loss: 0.2181, Train Acc: 90.61%, Test Loss: 0.2348, Test Acc: 89.90%
Epoch [50/50], Train Loss: 0.2170, Train Acc: 90.65%, Test Loss: 0.2360, Test Acc: 89.74%
Epoch [10/50], Train Lo

segment ini mendefinisikan parameter eksperimen:

- Hidden layers: Menguji dampak kedalaman jaringan
- Neurons: Memeriksa efek lebar jaringan
- Activations: Membandingkan fungsi non-linear berbeda
- Learning rates: Rentang dari sangat besar (10) hingga sangat kecil (0.0001)
- Batch sizes: Menguji ukuran batch pelatihan berbeda
- Epochs: Memvariasikan durasi pelatihan

Mendefinisikan konfigurasi dasar untuk perbandingan kemudiaMenyimpan hasil dalam format terstruktur yang Memungkinkan perbandingan sistematis arsitektur berbeda lalu Konfigurasi dasar menggunakan nilai-nilai yang umumnya efektif sebagai titik awal.
